<a href="https://colab.research.google.com/github/ychowdhrey/seo-scripts/blob/main/Meta%20%26%20Title%20Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Install necessary libraries
!pip install pandas openpyxl beautifulsoup4 tqdm

# STEP 2: Upload your Excel file
from google.colab import files
uploaded = files.upload()

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# STEP 3: Load the Excel file (replace with your filename if needed)
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)
urls = df.iloc[:, 0]  # Assuming URLs are in the first column

# STEP 4: Scrape metadata from each URL
results = []

for url in tqdm(urls):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.title.string.strip() if soup.title else ''
        meta_desc = soup.find('meta', attrs={'name': 'description'})
        og_title = soup.find('meta', property='og:title')
        og_desc = soup.find('meta', property='og:description')

        results.append({
            'URL': url,
            'Title': og_title['content'] if og_title else title,
            'Meta Description': og_desc['content'] if og_desc else (meta_desc['content'] if meta_desc else '')
        })
    except Exception as e:
        results.append({
            'URL': url,
            'Title': 'Error',
            'Meta Description': str(e)
        })

# STEP 5: Save results to Excel
output_df = pd.DataFrame(results)
output_filename = "article_metadata_output.xlsx"
output_df.to_excel(output_filename, index=False)

# STEP 6: Download the result
files.download(output_filename)
